In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

In [27]:
df_length = pd.read_csv('yandex_sti_w_price.csv', sep=';', encoding='utf-8')

In [30]:
df_length.head(3)

,line_id,route_number,type_ts,route_long_name,реестр_номер,реестр_имя,тип_тс,city,length,price,region
0,1704841444,1,bus,72-й квартал_Станция Новокуйбышевская,1,72-й квартал - Станция Новокуйбышевская,автобус,Новокуйбышевск,10.53,25,муниципальный
1,3245401927,1,bus,Автостанция Нефтегорск_Дмитриевка,1,г. Нефтегорск - с. Дмитриевка,автобус,Нефтегорский район,57.17,-,муниципальный
2,1704841008,1,bus,Автостанция_Автостанция,None,None,NaN,Похвистнево,19.89,-,муниципальный


In [31]:
np_dfl = df_length.to_numpy()
ind_tp = list(df_length.columns).index('type_ts')
ind_reg = list(df_length.columns).index('region')
ind_len = list(df_length.columns).index('length')
ind_prc = list(df_length.columns).index('price')
ind_ld = list(df_length.columns).index('line_id')

In [61]:
lst_min = []
lst_max = []
lst_prc_km = []
i=0
for i in tqdm(range(len(np_dfl))):
    tp_ts = np_dfl[i, ind_tp]
    reg = np_dfl[i, ind_reg]
    lngth = np_dfl[i, ind_len]
    lineid = np_dfl[i, ind_ld]
    
    try:
        prc_ttl = int(np_dfl[i, ind_prc])
    except:
        prc_ttl = np_dfl[i, ind_prc]    
    if tp_ts == 'water':
        mn_min = prc_ttl
        mn_max = prc_ttl
        prc_km = round(int(prc_ttl) / lngth, 1)
    elif tp_ts == 'suburban':
        mn_min = 80
        mn_max = prc_ttl
        prc_km = 2
    else:
        if 'муницип' in reg:
            if prc_ttl == '-':
                prc_ttl = 29
            mn_min = prc_ttl
            mn_max = prc_ttl
            prc_km = round(int(prc_ttl) / lngth, 1)
        else:
            if prc_ttl == '-':
                    prc_ttl = round(lngth * 3, 1)
            if prc_ttl < 60:
                mn_min = prc_ttl
                mn_max = 60
                prc_km = 3             
            else:
                mn_min = 60
                mn_max = round(lngth * 3, 1)
                prc_km = 3
    #
    lst_prc_km.append([lineid, mn_min, mn_max, prc_km])
# 

In [62]:
df_total_price = pd.DataFrame(lst_prc_km, columns=['line_id', 
                                                   'min_price', 'max_price', 'rub_km'])

In [63]:
df_total_price.head(2)

,line_id,min_price,max_price,rub_km
0,1704841444,25.0,25.0,2.4
1,3245401927,29.0,29.0,0.5


In [64]:
df_total_price.to_csv('df_total_price.csv', sep=';', encoding='utf-8-sig', index=False)